# 01 Сбор данных голосования
Сбор данных с сайта  UN по результатам голосования по конкретной резолюции

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

## 0.  Поисковые топики с сайта
https://digitallibrary.un.org/search?ln=ru&cc=Voting%20Data&p=&f=&rm=&ln=ru&sf=latest%20first&so=d&rg=50&c=Voting%20Data&c=&of=hb&fti=0&fct__9=Vote&fti=0

In [3]:
topic_year = '2022'
uri = 'https://digitallibrary.un.org/search?ln=ru&cc=Voting%20Data&p=&f=&rm=&ln=ru&sf=latest%20first&so=d&rg=50&c=Voting%20Data&c=&of=hb&fti=0&fct__9=Vote&fct__3=' + topic_year + '&fti=0'
r = requests.get(uri)
soup = BeautifulSoup(r.content, 'html.parser')
#print(soup.prettify())

In [628]:
#soup.findAll("ul", {"class": "option-fct"})[2].select("li")[0].findAll("span", {"class": ""})[0].text

In [629]:
#len(soup.findAll("ul", {"class": "option-fct"})[2].select("li"))

In [630]:
#soup.findAll("ul", {"class": "option-fct"})[2].select("li")[0]

In [4]:
topics_html = soup.findAll("ul", {"class": "option-fct"})[2].select("li")

topics_caption = []
topics_val = []

for index in range (0, len(topics_html)):
    topics_caption.append(topics_html[index].findAll("span", {"class": ""})[0].text)
    topics_val.append(topics_html[index].findAll("span", {"class": "pull-right"})[0].text)
    

topics_table = pd.DataFrame()
topics_table['Topics'] = topics_caption
topics_table['Val'] = topics_val
topics_table['Year'] = topic_year
topics_table['Val'] = topics_table['Val'].astype(int)

topics_table[0:21]

,Topics,Val,Year
0,HUMAN RIGHTS ADVANCEMENT,7,2022
1,LIBYA SITUATION,7,2022
2,SOMALIA SITUATION,6,2022
3,UKRAINE--POLITICAL CONDITIONS,6,2022
4,MIDDLE EAST SITUATION,4,2022
5,NUCLEAR NON-PROLIFERATION,4,2022
6,PALESTINE QUESTION,4,2022
7,SOUTH SUDAN--POLITICAL CONDITIONS,4,2022
8,SUDAN--POLITICAL CONDITIONS,4,2022
9,UN SUPPORT MISSION IN LIBYA,4,2022


In [9]:
topics_table[topics_table['Val'] >= 3]

,Topics,Val,Year
0,HUMAN RIGHTS ADVANCEMENT,7,2022
1,LIBYA SITUATION,7,2022
2,SOMALIA SITUATION,6,2022
3,UKRAINE--POLITICAL CONDITIONS,6,2022
4,MIDDLE EAST SITUATION,4,2022
5,NUCLEAR NON-PROLIFERATION,4,2022
6,PALESTINE QUESTION,4,2022
7,SOUTH SUDAN--POLITICAL CONDITIONS,4,2022
8,SUDAN--POLITICAL CONDITIONS,4,2022
9,UN SUPPORT MISSION IN LIBYA,4,2022


## 1. Скачиваем страницу голосования по резолюции

In [3]:
# Адрес страницы
uri = 'https://digitallibrary.un.org/record/4003921'


# поисковый топик на сайте
search_topic = 'UKRAINE--POLITICAL CONDITIONS'

#search_topic = 'PALESTINE QUESTION'
#search_topic = 'HUMAN RIGHTS--REPORTS'
tags = ['russia', 'ukraine', 'war']
#tags = ['Syrian']

r = requests.get(uri)
soup = BeautifulSoup(r.content, 'html.parser')

In [81]:
# print(soup.prettify())

## 2. Получаем основной блок html

In [4]:
html_block = soup.findAll("div", {"id": "details-collapse"})
print(len(html_block))
#print()
#print(html_block[0].prettify())

1


### 2.1 Получаем заголовок резолюции

In [5]:
result = html_block[0].select("div")

In [6]:
print(result[0].select("span")[1].text)

Principles of the Charter of the United Nations underlying a comprehensive, just and lasting peace in Ukraine : resolution / adopted by the General Assembly


### 2.2 Получаем идентификатор резолюции

In [7]:
resolution_no = result[2].select("span")[1].text
resolution_no

'A/RES/ES-11/6'

In [86]:
# Ссылка на резолюцию
#result[2].select("span")[1].findAll("a", {})[0].attrs['href']

### 2.3 Проверяем на дубликат

In [8]:
# Читаем общую таблицу по голосованиям с диска
votes_table_com = pd.read_csv('votes_table.csv')
votes_table_com

,Data,Resolution,Title,Uri,Tags,Topic
0,2022-11-14,A/RES/ES-11/5,Furtherance of remedy and reparation for aggre...,https://digitallibrary.un.org/record/3994052,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
1,2022-10-12,A/RES/ES-11/4,Territorial integrity of Ukraine : defending t...,https://digitallibrary.un.org/record/3990400,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
2,2022-04-07,A/RES/ES-11/3,Suspension of the rights of membership of the ...,https://digitallibrary.un.org/record/3967778,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
3,2022-03-24,A/RES/ES-11/2,Humanitarian consequences of the aggression ag...,https://digitallibrary.un.org/record/3965954,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
4,2022-03-02,A/RES/ES-11/1,Aggression against Ukraine : resolution / adop...,https://digitallibrary.un.org/record/3959039,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
5,2022-11-30,A/RES/77/22,Committee on the Exercise of the Inalienable R...,https://digitallibrary.un.org/record/3996096,"['Palestine, Israel', 'war']",PALESTINE QUESTION
6,2022-11-30,A/RES/77/23,Division for Palestinian Rights of the Secreta...,https://digitallibrary.un.org/record/3996095,"['Palestine, Israel', 'war']",PALESTINE QUESTION
7,2022-11-30,A/RES/77/25,Peaceful settlement of the question of Palesti...,https://digitallibrary.un.org/record/3996093,"['Palestine, Israel', 'war']",PALESTINE QUESTION
8,2022-11-30,A/RES/77/24,Special information programme on the question ...,https://digitallibrary.un.org/record/3996092,"['Palestine, Israel', 'war']",PALESTINE QUESTION
9,2022-12-15,A/RES/77/228,Situation of human rights in the Islamic Repub...,https://digitallibrary.un.org/record/3997720,['Iran'],HUMAN RIGHTS--REPORTS


In [9]:
if len(votes_table_com[votes_table_com['Resolution'] == result[2].select("span")[1].text]) > 0:
    print('Дубликат по номеру резолюции')
else: 
    print('Новая резолюция') 

Новая резолюция


### 2.4 Получаем подробные результаты голосования и помещаем в DataFrame

In [23]:
#print(votes_resalt_detail[1].contents)
#result[10] 5 8 11
voite_result_bs = result[9]
voite_result_bs

<div class="metadata-row"><span class="title col-xs-12 col-sm-3 col-md-2">Vote





</span><span class="value col-xs-12 col-sm-9 col-md-10">Y AFGHANISTAN<br/> Y ALBANIA<br/> A ALGERIA<br/> Y ANDORRA<br/> A ANGOLA<br/> Y ANTIGUA AND BARBUDA<br/> Y ARGENTINA<br/> A ARMENIA<br/> Y AUSTRALIA<br/> Y AUSTRIA<br/>  AZERBAIJAN<br/> Y BAHAMAS<br/> Y BAHRAIN<br/> A BANGLADESH<br/> Y BARBADOS<br/> N BELARUS<br/> Y BELGIUM<br/> Y BELIZE<br/> Y BENIN<br/> Y BHUTAN<br/> A BOLIVIA (PLURINATIONAL STATE OF)<br/> Y BOSNIA AND HERZEGOVINA<br/> Y BOTSWANA<br/> Y BRAZIL<br/> Y BRUNEI DARUSSALAM<br/> Y BULGARIA<br/>  BURKINA FASO<br/> A BURUNDI<br/> Y CABO VERDE<br/> Y CAMBODIA<br/>  CAMEROON<br/> Y CANADA<br/> A CENTRAL AFRICAN REPUBLIC<br/> Y CHAD<br/> Y CHILE<br/> A CHINA<br/> Y COLOMBIA<br/> Y COMOROS<br/> A CONGO<br/> Y COSTA RICA<br/> Y COTE D'IVOIRE<br/> Y CROATIA<br/> A CUBA<br/> Y CYPRUS<br/> Y CZECHIA<br/> N DEMOCRATIC PEOPLE'S REPUBLIC OF KOREA<br/> Y DEMOCRATIC REPUBLIC OF THE CONGO<br/> Y DENMA

In [24]:
#voite_result_bs.select("span")[1].contents
voite_result_bs.select("span")[1].contents

['Y AFGHANISTAN',
 <br/>,
 ' Y ALBANIA',
 <br/>,
 ' A ALGERIA',
 <br/>,
 ' Y ANDORRA',
 <br/>,
 ' A ANGOLA',
 <br/>,
 ' Y ANTIGUA AND BARBUDA',
 <br/>,
 ' Y ARGENTINA',
 <br/>,
 ' A ARMENIA',
 <br/>,
 ' Y AUSTRALIA',
 <br/>,
 ' Y AUSTRIA',
 <br/>,
 '  AZERBAIJAN',
 <br/>,
 ' Y BAHAMAS',
 <br/>,
 ' Y BAHRAIN',
 <br/>,
 ' A BANGLADESH',
 <br/>,
 ' Y BARBADOS',
 <br/>,
 ' N BELARUS',
 <br/>,
 ' Y BELGIUM',
 <br/>,
 ' Y BELIZE',
 <br/>,
 ' Y BENIN',
 <br/>,
 ' Y BHUTAN',
 <br/>,
 ' A BOLIVIA (PLURINATIONAL STATE OF)',
 <br/>,
 ' Y BOSNIA AND HERZEGOVINA',
 <br/>,
 ' Y BOTSWANA',
 <br/>,
 ' Y BRAZIL',
 <br/>,
 ' Y BRUNEI DARUSSALAM',
 <br/>,
 ' Y BULGARIA',
 <br/>,
 '  BURKINA FASO',
 <br/>,
 ' A BURUNDI',
 <br/>,
 ' Y CABO VERDE',
 <br/>,
 ' Y CAMBODIA',
 <br/>,
 '  CAMEROON',
 <br/>,
 ' Y CANADA',
 <br/>,
 ' A CENTRAL AFRICAN REPUBLIC',
 <br/>,
 ' Y CHAD',
 <br/>,
 ' Y CHILE',
 <br/>,
 ' A CHINA',
 <br/>,
 ' Y COLOMBIA',
 <br/>,
 ' Y COMOROS',
 <br/>,
 ' A CONGO',
 <br/>,
 ' Y COSTA RICA'

In [25]:
#[x for x in voite_result_bs.select("span")[1].contents if getattr(x, 'name', None) != 'br']
if (len(voite_result_bs.select("span")[1].contents) == 2):
    votes_resalt_detail_raw_arr = [x for x in voite_result_bs.select("span")[1].contents[1] if getattr(x, 'name', None) != 'br']
    votes_resalt_detail_raw_arr.insert(0, voite_result_bs.select("span")[1].contents[0])
else: votes_resalt_detail_raw_arr = [x for x in voite_result_bs.select("span")[1].contents if getattr(x, 'name', None) != 'br']
    
votes_resalt_detail_raw_arr

['Y AFGHANISTAN',
 ' Y ALBANIA',
 ' A ALGERIA',
 ' Y ANDORRA',
 ' A ANGOLA',
 ' Y ANTIGUA AND BARBUDA',
 ' Y ARGENTINA',
 ' A ARMENIA',
 ' Y AUSTRALIA',
 ' Y AUSTRIA',
 '  AZERBAIJAN',
 ' Y BAHAMAS',
 ' Y BAHRAIN',
 ' A BANGLADESH',
 ' Y BARBADOS',
 ' N BELARUS',
 ' Y BELGIUM',
 ' Y BELIZE',
 ' Y BENIN',
 ' Y BHUTAN',
 ' A BOLIVIA (PLURINATIONAL STATE OF)',
 ' Y BOSNIA AND HERZEGOVINA',
 ' Y BOTSWANA',
 ' Y BRAZIL',
 ' Y BRUNEI DARUSSALAM',
 ' Y BULGARIA',
 '  BURKINA FASO',
 ' A BURUNDI',
 ' Y CABO VERDE',
 ' Y CAMBODIA',
 '  CAMEROON',
 ' Y CANADA',
 ' A CENTRAL AFRICAN REPUBLIC',
 ' Y CHAD',
 ' Y CHILE',
 ' A CHINA',
 ' Y COLOMBIA',
 ' Y COMOROS',
 ' A CONGO',
 ' Y COSTA RICA',
 " Y COTE D'IVOIRE",
 ' Y CROATIA',
 ' A CUBA',
 ' Y CYPRUS',
 ' Y CZECHIA',
 " N DEMOCRATIC PEOPLE'S REPUBLIC OF KOREA",
 ' Y DEMOCRATIC REPUBLIC OF THE CONGO',
 ' Y DENMARK',
 ' Y DJIBOUTI',
 '  DOMINICA',
 ' Y DOMINICAN REPUBLIC',
 ' Y ECUADOR',
 ' Y EGYPT',
 ' A EL SALVADOR',
 '  EQUATORIAL GUINEA',
 ' N 

In [26]:
votes_resalt_detail_arr_res = []
votes_resalt_detail_arr_country = []

for item in [x for x in votes_resalt_detail_raw_arr]:
    
    if item[0:12] == ' AFGHANISTAN':
        votes_resalt_detail_arr_res.append(None)
        votes_resalt_detail_arr_country.append(item[1:])
        
    if item[0:2] == 'Y ':
        votes_resalt_detail_arr_res.append(1)
        votes_resalt_detail_arr_country.append(item[2:])
        
    if item[0:2] == 'A ':
        votes_resalt_detail_arr_res.append(0)
        votes_resalt_detail_arr_country.append(item[2:])
        
    if item[0:2] == 'N ':
        votes_resalt_detail_arr_res.append(-1)
        votes_resalt_detail_arr_country.append(item[2:])
        
    if item[0:2] == '  ':
        votes_resalt_detail_arr_res.append(None)  
        votes_resalt_detail_arr_country.append(item[2:])
        
    if item[0:3] == ' Y ':
        votes_resalt_detail_arr_res.append(1)    
        votes_resalt_detail_arr_country.append(item[3:])
        
    if item[0:3] == ' A ':
        votes_resalt_detail_arr_res.append(0)    
        votes_resalt_detail_arr_country.append(item[3:])
        
    if item[0:3] == ' N ':
        votes_resalt_detail_arr_res.append(-1)    
        votes_resalt_detail_arr_country.append(item[3:])

In [27]:
votes_resalt_detail_table = pd.DataFrame()

votes_resalt_detail_table['Country'] = votes_resalt_detail_arr_country
votes_resalt_detail_table[resolution_no] = votes_resalt_detail_arr_res

votes_resalt_detail_table

,Country,A/RES/ES-11/6
0,AFGHANISTAN,1.0
1,ALBANIA,1.0
2,ALGERIA,0.0
3,ANDORRA,1.0
4,ANGOLA,0.0
...,...,...
188,VENEZUELA (BOLIVARIAN REPUBLIC OF),NaN
189,VIET NAM,0.0
190,YEMEN,1.0
191,ZAMBIA,1.0


In [29]:
#4 6 7
print(result[5].select("div")[1].text)
#result[4].select("div")

Yes: 141 | No: 7 | Abstentions: 32 | Non-Voting: 13 | Total voting membership: 193 


In [30]:
print('Yes: {} | No: {} | Abstentions: {} | Non-Voting: {} | Total voting membership: {}'
      .format(len(votes_resalt_detail_table[votes_resalt_detail_table[resolution_no] == 1]),
              len(votes_resalt_detail_table[votes_resalt_detail_table[resolution_no] == -1]),
              len(votes_resalt_detail_table[votes_resalt_detail_table[resolution_no] == 0]),
              len(votes_resalt_detail_table[pd.isna(votes_resalt_detail_table[resolution_no])]),
              len(votes_resalt_detail_table)
      ))

Yes: 141 | No: 7 | Abstentions: 32 | Non-Voting: 13 | Total voting membership: 193


## 3. Записываем таблицу результатов в файл csv

In [31]:
votes_resalt_detail_table.to_csv('resolution_vote/' + resolution_no.replace("/", "_") + '.csv', index=False)  

## 4. Добавляем сведения о голосовании в сводную таблицу

### 4.1 Создаем таблицу для текущего голосования

In [33]:
# 8 10
voite_data = result[8].select("span")[1].text
voite_data

'2023-02-23'

In [98]:
#result[9]

In [34]:
votes_table = pd.DataFrame()
#votes_table['Data'] = pd.to_datetime([voite_data[1].text], format='%Y-%m-%d')
votes_table['Data'] = [voite_data]
votes_table['Resolution'] = [result[2].select("span")[1].text]
votes_table['Title'] = [result[0].select("span")[1].text]
votes_table['Uri'] = uri
votes_table['Tags'] = [tags] # is not array!
votes_table['Topic'] = [search_topic]

#votes_table.to_csv('votes_table.csv', index=False)  
votes_table

,Data,Resolution,Title,Uri,Tags,Topic
0,2023-02-23,A/RES/ES-11/6,Principles of the Charter of the United Nation...,https://digitallibrary.un.org/record/4003921,"[russia, ukraine, war]",UKRAINE--POLITICAL CONDITIONS


### 4.3 Добавляем новую запись в общую таблицу

In [100]:
# Проверка на дебликат
#if len(votes_table_com[votes_table_com['Resolution'] == votes_table['Resolution']]) > 0:
    #print('Дубликат по номеру резолюции')
#else: 
    #votes_table_com = pd.concat([votes_table_com, votes_table])
    #print('Новая резолюция добавлена в таблицу') 

In [35]:
votes_table_com = pd.concat([votes_table_com, votes_table])
votes_table_com

,Data,Resolution,Title,Uri,Tags,Topic
0,2022-11-14,A/RES/ES-11/5,Furtherance of remedy and reparation for aggre...,https://digitallibrary.un.org/record/3994052,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
1,2022-10-12,A/RES/ES-11/4,Territorial integrity of Ukraine : defending t...,https://digitallibrary.un.org/record/3990400,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
2,2022-04-07,A/RES/ES-11/3,Suspension of the rights of membership of the ...,https://digitallibrary.un.org/record/3967778,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
3,2022-03-24,A/RES/ES-11/2,Humanitarian consequences of the aggression ag...,https://digitallibrary.un.org/record/3965954,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
4,2022-03-02,A/RES/ES-11/1,Aggression against Ukraine : resolution / adop...,https://digitallibrary.un.org/record/3959039,"['russia', 'ukraine', 'war']",UKRAINE--POLITICAL CONDITIONS
5,2022-11-30,A/RES/77/22,Committee on the Exercise of the Inalienable R...,https://digitallibrary.un.org/record/3996096,"['Palestine, Israel', 'war']",PALESTINE QUESTION
6,2022-11-30,A/RES/77/23,Division for Palestinian Rights of the Secreta...,https://digitallibrary.un.org/record/3996095,"['Palestine, Israel', 'war']",PALESTINE QUESTION
7,2022-11-30,A/RES/77/25,Peaceful settlement of the question of Palesti...,https://digitallibrary.un.org/record/3996093,"['Palestine, Israel', 'war']",PALESTINE QUESTION
8,2022-11-30,A/RES/77/24,Special information programme on the question ...,https://digitallibrary.un.org/record/3996092,"['Palestine, Israel', 'war']",PALESTINE QUESTION
9,2022-12-15,A/RES/77/228,Situation of human rights in the Islamic Repub...,https://digitallibrary.un.org/record/3997720,['Iran'],HUMAN RIGHTS--REPORTS


### 4.4 Сохраняем общую таблицу на диск

In [36]:
votes_table_com.to_csv('votes_table.csv', index=False) 